[View in Colaboratory](https://colab.research.google.com/github/rkhaled98/Karmalutional-Network/blob/master/scripts/KARMALUTIONALNETWORKKERAS.ipynb)

In [9]:
from google.colab import files
uploaded = files.upload()
# you have to download this file and upload it (unzip too): https://www.kaggle.com/devjyotichandra/glove6b50dtxt
# make sure to change the name to glove.6B.50d.txt

Saving glove.6B.50d.txt to glove.6B.50d.txt


In [93]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2018-05-11 19:28:55--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2018-05-11 19:28:55--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip   12%[=>                  ] 231.60M  7.05MB/s    eta 3m 17s 

glove.42B.300d.zip  100%[===================>]   1.75G  10.5MB/s    in 4m 19s  

2018-05-11 19:33:14 (6.91 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]



In [94]:
!unzip glove.42B.300d.zip

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [81]:
from google.colab import files
uploaded = files.upload()
# get these files exactly from git repo under data folder: data/news_news_small_1.csv and data/news_news_small_test_1.csv


In [24]:
!ls

datalab		    glove.6B.50d.txt	      news_news_small_test_1 (1).csv
glove.42B.300d.txt  news_news_small_1.csv     news_news_small_test_1.csv
glove.42B.300d.zip  news_news_small.outfinal


In [10]:
!ls # make sure there are the files required above in the folder

datalab		    glove.6B.50d.txt	      news_news_small_test_1 (1).csv
glove.42B.300d.txt  news_news_small_1.csv     news_news_small_test_1.csv
glove.42B.300d.zip  news_news_small.outfinal


In [0]:
import csv
import numpy as np

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
  
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
  
def read_csv(filename = 'data/news_news_small_1.csv'):
    phrase = []
    rank = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            # print(phrase)
            rank.append(row[1])

    X = np.asarray(phrase)
    # print(X)
    Y = np.asarray(rank)

    return X, Y
  
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [0]:
def predict(X, Y, W, b, word_to_vec_map, output_prediction=False):
    """
    Given X (sentences) and Y (relative score value), predict the Y and get the accuracy.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing relative score index, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()
        
        # Average words' vectors
        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
        if output_prediction:
           print("The model predicts that the comment %s will receive a relative score ranking of %s" % (X[j], pred[j]))
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred

In [0]:
def predict_one(X, W, b, word_to_vec_map):
    words = X.lower().split()
    avg = np.zeros((50,))
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg/len(words)
    Z = np.dot(W, avg) + b
    A = softmax(Z)
    pred = np.argmax(A)
    print("The model predicts that the comment: \n'%s'\n will receive a relative score ranking of: %s" % (X, pred))

In [28]:
print("reading training data")
X_train, Y_train = read_csv('news_news_small_1.csv') # the data needed by the read_csv is a csv with comment,rank generated by the datasets main script 
print("done reading training data")
filtered = list(filter(None, Y_train))
Y_train = np.asarray(filtered)
Y_train = Y_train.astype(int)
#print(Y_train) make sure theyre ints
print("done reading testing data")
X_test, Y_test = read_csv('news_news_small_test_1.csv')
print("done reading testing data")
filtered = list(filter(None, Y_test))
Y_test = np.asarray(filtered)
Y_test = Y_test.astype(int)
#print(Y_test) make sure theyre ints

# Y_train = np.append(Y_train, [0])
print(X_train.shape, Y_train.shape)
# Y_test = np.append(Y_test, [0])
print(X_test.shape, Y_test.shape)

reading training data
done reading training data
done reading testing data
done reading testing data
(3852,) (3852,)
(442,) (442,)


In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [30]:
print("reading glove vector")
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')
print("done reading glove vector")

reading glove vector
done reading glove vector


In [0]:
def remove_non_keys(X, dict):
    for i in range(0, len(X)):
        for word in X[i].split(" "):
            if (not (word in dict)):
                if (not (word == " ")):
                    if (not (word == "")):
                        X[i] = X[i].replace(word, '')
                        X[i] = " ".join(X[i].split())
    return X

In [32]:
print("removing non keys")
X_train = remove_non_keys(X_train, word_to_index)
X_train = remove_non_keys(X_train, word_to_index)
X_test = remove_non_keys(X_test, word_to_index)
X_test = remove_non_keys(X_test, word_to_index)
print("done removing non keys")

removing non keys
done removing non keys


In [0]:
def sentence_to_avg(sentence, word_to_vec_map):
  
    # MAKE SURE TO CHANGE THE DIMENSIONS IN HERE IF YOU CHANGE THE GLOVE VECTOR
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    #Split sentence into list of lower case words (around 1 line)
    words = [i.lower() for i in sentence.split()]

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros((50,))
    
    #average the word vectors. You can loop over the words in the list "words".
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg / len(words)
    
    return avg

In [0]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    """
    Model to train word vector representations in numpy.
    
    Arguments:
    X -- input data, numpy array of comment sentences as strings, of shape (m, 1)
    Y -- labels, numpy array of integers between 0 and 9, numpy-array of shape (m, 1)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    learning_rate -- learning_rate for the stochastic gradient descent algorithm
    num_iterations -- number of iterations
    
    Returns:
    pred -- vector of predictions, numpy-array of shape (m, 1)
    W -- weight matrix of the softmax layer, of shape (n_y, n_h)
    b -- bias of the softmax layer, of shape (n_y,)
    """
    
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 10                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    alpha,hidden_dim,dropout_percent,do_dropout = (0.5,10,0.1,False) # playing around with dropout
    
    # Optimization loop
    for t in range(num_iterations):                       # Loop over the number of iterations
        for i in range(m):                                # Loop over the training examples
            
            # Average the word vectors of the words from the i'th training example
            avg = sentence_to_avg(X[i], word_to_vec_map)

            # Forward propagate the avg through the softmax layer
            z = np.dot(W, avg) + b
            dropout = np.random.binomial([np.ones((hidden_dim))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))
            if do_dropout:
                z *= dropout
            a = softmax(z)

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -np.sum(np.multiply(Y_oh[i], np.log(a)))
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 20 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            #W *= 
#             print(z)
#             print(dropout.shape)
#             print(a)
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

In [75]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

Epoch: 0 --- cost = 2.4302035956213324
Accuracy: 0.26479750778816197
Epoch: 20 --- cost = 2.46999121077523
Accuracy: 0.3060747663551402
Epoch: 40 --- cost = 2.450790465228912
Accuracy: 0.32217030114226375
Epoch: 60 --- cost = 2.428775119293687
Accuracy: 0.3260643821391485
Epoch: 80 --- cost = 2.4091642175206625
Accuracy: 0.3258047767393562
Epoch: 100 --- cost = 2.392738949893456
Accuracy: 0.3247663551401869
Epoch: 120 --- cost = 2.378723057059636
Accuracy: 0.3273624091381101
Epoch: 140 --- cost = 2.366227502872948
Accuracy: 0.32320872274143303
Epoch: 160 --- cost = 2.3546177093065435
Accuracy: 0.3242471443406023
Epoch: 180 --- cost = 2.3435122605420635
Accuracy: 0.3242471443406023
Epoch: 200 --- cost = 2.3327055222155706
Accuracy: 0.32502596053997923
Epoch: 220 --- cost = 2.322098169044641
Accuracy: 0.3258047767393562
Epoch: 240 --- cost = 2.3116499262032573
Accuracy: 0.32528556593977154
Epoch: 260 --- cost = 2.3013508147566446
Accuracy: 0.3263239875389408
Epoch: 280 --- cost = 2.29120

In [76]:
predict_one("Rafi is the best", W, b, word_to_vec_map)
predict_one("Nanak is the best", W, b, word_to_vec_map)
predict_one("Colby is the best", W, b, word_to_vec_map)
predict_one("Brian is the best", W, b, word_to_vec_map)
predict_one("Kevin is the best but Rafi is better", W, b, word_to_vec_map)
predict_one("Victor is the best", W, b, word_to_vec_map)
predict_one("warrior princess is the best", W, b, word_to_vec_map)
# predict_one("Victor is the greatest of all time", W, b, word_to_vec_map)

The model predicts that the comment: 
'Rafi is the best'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Nanak is the best'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Colby is the best'
 will receive a relative score ranking of: 1
The model predicts that the comment: 
'Brian is the best'
 will receive a relative score ranking of: 8
The model predicts that the comment: 
'Kevin is the best but Rafi is better'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Victor is the best'
 will receive a relative score ranking of: 5
The model predicts that the comment: 
'warrior princess is the best'
 will receive a relative score ranking of: 8


In [77]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

Using TensorFlow backend.


In [0]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [w.lower() for w in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j += 1
            
    ### END CODE HERE ###
    
    return X_indices

In [79]:

X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [0]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [81]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


In [0]:
# GRADED FUNCTION: Emojify_V2

def Model_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Model-V2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(10)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
   
    
    return model

In [0]:
maxLen = len(max(X_train, key=len).split())

In [97]:
model = Model_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 506)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 506, 50)           20000050  
_________________________________________________________________
lstm_5 (LSTM)                (None, 506, 128)          91648     
_________________________________________________________________
dropout_5 (Dropout)          (None, 506, 128)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
__________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 10)

In [0]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
3852/3852 [==============================] - 294s 76ms/step - loss: 2.1536 - acc: 0.2591
Epoch 2/50
3852/3852 [==============================] - 294s 76ms/step - loss: 2.1205 - acc: 0.2658
Epoch 3/50
3852/3852 [==============================] - 296s 77ms/step - loss: 2.1185 - acc: 0.2651
Epoch 4/50
2336/3852 [=================>............] - ETA: 1:55 - loss: 2.1151 - acc: 0.2693

3852/3852 [==============================] - 294s 76ms/step - loss: 2.1194 - acc: 0.2658
Epoch 5/50
3852/3852 [==============================] - 295s 76ms/step - loss: 2.1118 - acc: 0.2658
Epoch 6/50
2880/3852 [=====================>........] - ETA: 1:13 - loss: 2.1162 - acc: 0.2660